In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [42]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Convert to lower case
    tokens = [token.lower() for token in tokens]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Stem the tokens
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)
    
    return preprocessed_text


exampl sentenc demonstr text preprocess nltk .


In [3]:
from langdetect import detect

# Function to detect the language of text
def detect_language(text):
    try:
        return detect(text)
    except:
        return print("Error cannot detect language")

# load dataset

In [ ]:
hermes_process_state = pd.read_csv('dataset/hermes_process_states.csv')
hermes_process_state.head()

In [ ]:
knowledge_tools = pd.read_csv('dataset/knowledge_tools.csv')
knowledge_tools.head()

In [ ]:
knowledge_analysis_categories = pd.read_csv('dataset/knowledge_analysis_categories.csv')

knowledge_analysis_categories.sort_values(by='escalation_status', ascending=False, inplace=True)
knowledge_analysis_categories

In [ ]:
hermes_process_logs = pd.read_csv('dataset/hermes_process_logs.csv')
hermes_process_logs.head()

In [ ]:
hermes_process_logs.info()

In [ ]:
# check null values
hermes_process_logs.isnull().sum()

In [ ]:
# drop all null value
hermes_process_logs.dropna(inplace=True)

In [ ]:
# categoory count
hermes_process_logs['category_name'].value_counts()

In [ ]:
# count anget_name
hermes_process_logs['agent_name'].value_counts()

In [ ]:
# delete other agent_name except 'Hosting'
hermes_process_logs = hermes_process_logs[hermes_process_logs['agent_name'] == 'Hosting']

hermes_process_logs['category_name'].value_counts()

In [ ]:
hermes_process_logs['state_name'].value_counts()

## filter by category

In [ ]:
# Step 1: Make a copy of the original DataFrame
hermes_process_copy = hermes_process_logs.copy()

# Step 2: Drop rows where 'category_name' starts with 'query'
hermes_process_copy = hermes_process_copy[~hermes_process_copy['category_name'].str.startswith('query')]

# Step 3: Drop rows where 'category_name' starts with 'information'
hermes_process_copy = hermes_process_copy[~hermes_process_copy['category_name'].str.startswith('information')]

# Step 4: Check unique 'category_name' values in the resulting DataFrame
unique_category_names = hermes_process_copy['category_name'].unique()
unique_category_names

In [ ]:
# check form the data on last message if it contain the word 'request'
request = hermes_process_copy['category_name'].str.contains('request').sum()
print(request ==len(hermes_process_copy))

In [ ]:
# new dataframe contain only last message and category_name
message = hermes_process_logs[['last_message', 'category_name']]

In [ ]:
# drop duplicate message for the last message
message.duplicated().sum()
message = message.drop_duplicates().copy()

In [ ]:
message['category_name'].value_counts()

In [ ]:
# Apply preprocessing to 'last_message' column
message['last_message_clean'] = message['last_message'].apply(preprocess_text)

# Drop messages without any alphabet characters
message = message[message['last_message_clean'].str.match('.*[a-zA-Z].*')]

# Reset index after dropping rows
message.reset_index(drop=True, inplace=True)

# Display the preprocessed DataFrame
message.head()


In [ ]:
# Extracting features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(message['last_message_clean'])

In [ ]:
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=message['category_name'])

# Group by category and sum the TF-IDF scores for each word
tfidf_sums = tfidf_df.groupby(tfidf_df.index).sum()

In [ ]:
# Get the top keywords for each category
top_keywords = {}
for category in tfidf_sums.index:
    top_keywords[category] = tfidf_sums.loc[category].sort_values(ascending=False).head(5).index.tolist()

# Display top keywords for each category
for category, keywords in top_keywords.items():
    print(f"Category: {category}")
    print("Top keywords:", ", ".join(keywords))

# load new clean dataset

In [34]:
# load request csv into dataframe
request = pd.read_csv('dataset/request.csv')
request.head() 

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_name,created_at,updated_at,information
0,13343,HMDKXDNERP,Hosting,baik ..\nterimakasih kak,"Halo Larasati,\n\nTerima kasih telah menginfor...",2.0,request_guest_size_alteration,Larasati Dewi informed that there will be 5 fe...,Confirm the number of guests and ensure the sm...,get_guest_size_alteration_information,2024-04-23 00:04:25,2024-04-23 00:05:13,Ask the guest and confirm about additional gue...
1,13354,HM34KQ5BHD,Hosting,Merci beaucoup c’est très gentil de votre part...,"Bonjour Shirly,\n\nIt's truly a pleasure, and ...",2.0,request_early_check_in,Shirly's friend is planning to arrive early to...,Acknowledge Shirly's request for her friend to...,get_property_information,2024-04-23 00:35:11,2024-04-23 00:35:44,"EARLY CHECK-IN\nOur check-in time is at 2 PM, ..."
2,13354,HM34KQ5BHD,Hosting,Merci beaucoup c’est très gentil de votre part...,"Bonjour Shirly,\n\nIt's truly a pleasure, and ...",2.0,request_early_check_in,Shirly's friend is planning to arrive early to...,Acknowledge Shirly's request for her friend to...,send_arrival_information_to_staff,2024-04-23 00:35:11,2024-04-23 00:35:44,Message was not sent because it has been sent ...
3,13357,HM43CK89JC,Hosting,"Hi, there’s ants in the number 3 bathroom and ...","Hi Bailey,\n\nI'm sorry to hear about the issu...",1.0,request_onsite,Bailey Heap reported encountering ants in the ...,Assist with on-site issues regarding ant prese...,send_onsite_request_to_staff,2024-04-23 00:42:36,2024-04-23 00:43:17,Message has been sent to staff. Message detail...
4,13363,HMHWNXDWSD,Hosting,Hallo leider kam niemand im mich abzuholen. Ic...,"Dear Laura,\n\nI'm contacting our on-site staf...",1.0,request_onsite,Laura is inquiring about the absence of the pi...,Address Laura's concern about the missed pick-...,send_onsite_request_to_staff,2024-04-23 00:54:17,2024-04-23 00:54:49,Message was not sent because it has been sent ...


In [35]:
# detect language of the of last message
request['language'] = request['last_message'].apply(detect_language)

In [36]:
result = request.groupby(['language', 'category_name']).size().reset_index(name='count')
# Pivot the table to get languages as rows, category names as columns, and counts as values
pivot_table = result.pivot_table(index='language', columns='category_name', values='count', fill_value=0)
pivot_table


category_name,request_cancellation,request_early_check_in,request_extension,request_extra_service,request_guest_size_alteration,request_late_check_in,request_late_check_out,request_luggage_drop,request_onsite,request_property_review_summary,request_scooter_or_bike,request_shorten_alteration,request_special_requirements
language,,,,,,,,,,,,,
af,0,2,0,2,0,7,0,0,22,0,4,0,0
ca,0,0,0,2,0,3,0,0,7,0,0,0,0
cs,0,0,0,0,0,1,0,0,8,0,0,0,0
cy,0,0,0,0,0,0,0,0,2,0,0,0,0
da,0,3,0,0,0,0,0,0,3,11,4,0,0
de,0,9,0,11,0,36,0,0,30,0,24,2,2
en,3,267,11,58,2,539,8,18,1396,38,246,15,91
es,0,4,0,0,0,9,0,0,22,0,2,0,0
et,0,0,0,0,0,0,0,0,5,0,2,0,0


In [43]:
# only keep the rows where the 'language' column is 'en'
english_requests = request[request['language'] == 'en']
english_requests.head()

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_name,created_at,updated_at,information,language
3,13357,HM43CK89JC,Hosting,"Hi, there’s ants in the number 3 bathroom and ...","Hi Bailey,\n\nI'm sorry to hear about the issu...",1.0,request_onsite,Bailey Heap reported encountering ants in the ...,Assist with on-site issues regarding ant prese...,send_onsite_request_to_staff,2024-04-23 00:42:36,2024-04-23 00:43:17,Message has been sent to staff. Message detail...,en
6,13370,HMTJDQMSS2,Hosting,"Hi, i would like to know if you can make scoot...","Hi Maelle,\n\nAbsolutely, we can assist with s...",1.0,request_scooter_or_bike,Maelle is requesting scooter rentals from this...,Provide information and assistance for scooter...,get_property_information,2024-04-23 01:07:26,2024-04-23 01:09:33,• ADDITIONAL SERVICE \n ◦ Airport Pick Up ...,en
7,13370,HMTJDQMSS2,Hosting,"Hi, i would like to know if you can make scoot...","Hi Maelle,\n\nAbsolutely, we can assist with s...",1.0,request_scooter_or_bike,Maelle is requesting scooter rentals from this...,Provide information and assistance for scooter...,send_scooter_request_to_staff,2024-04-23 01:07:26,2024-04-23 01:09:33,Message has been sent to staff. Message detail...,en
8,13375,HMBN5QN43Z,Hosting,"Hello, do you currently have someone of the st...","Hi Vladislav,\n\nI'm really sorry to hear abou...",1.0,request_onsite,Vladislav reported a ceiling leak and flooding...,Address the ceiling leak and flooding issue at...,send_onsite_request_to_staff,2024-04-23 01:14:21,2024-04-23 01:15:02,Message has been sent to staff. Message detail...,en
9,13378,HMAYTSY224,Hosting,"Hello, could you give me the key from garage p...","Hi Roman,\n\nYour request for the garage key h...",1.0,request_onsite,The guest is requesting the key from the garag...,Assist in providing access to the garage key,send_onsite_request_to_staff,2024-04-23 01:16:53,2024-04-23 01:17:15,Message has been sent to staff. Message detail...,en


In [8]:
english_requests['category_name'].value_counts()

category_name
request_onsite                     1399
request_late_check_in               539
request_early_check_in              266
request_scooter_or_bike             245
request_special_requirements         90
request_extra_service                57
request_property_review_summary      39
request_luggage_drop                 19
request_shorten_alteration           15
request_extension                    11
request_late_check_out                8
request_cancellation                  3
request_guest_size_alteration         2
Name: count, dtype: int64

In [24]:
onsite_requests = english_requests[english_requests['category_name'] == 'request_onsite']

In [25]:
onsite_requests.head()

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_name,created_at,updated_at,information,language
3,13357,HM43CK89JC,Hosting,"Hi, there’s ants in the number 3 bathroom and ...","Hi Bailey,\n\nI'm sorry to hear about the issu...",1.0,request_onsite,Bailey Heap reported encountering ants in the ...,Assist with on-site issues regarding ant prese...,send_onsite_request_to_staff,2024-04-23 00:42:36,2024-04-23 00:43:17,Message has been sent to staff. Message detail...,en
8,13375,HMBN5QN43Z,Hosting,"Hello, do you currently have someone of the st...","Hi Vladislav,\n\nI'm really sorry to hear abou...",1.0,request_onsite,Vladislav reported a ceiling leak and flooding...,Address the ceiling leak and flooding issue at...,send_onsite_request_to_staff,2024-04-23 01:14:21,2024-04-23 01:15:02,Message has been sent to staff. Message detail...,en
9,13378,HMAYTSY224,Hosting,"Hello, could you give me the key from garage p...","Hi Roman,\n\nYour request for the garage key h...",1.0,request_onsite,The guest is requesting the key from the garag...,Assist in providing access to the garage key,send_onsite_request_to_staff,2024-04-23 01:16:53,2024-04-23 01:17:15,Message has been sent to staff. Message detail...,en
27,13465,HM4KT25MP5,Hosting,Selamat Pagi\n\nYes I would like to book trans...,"Selamat Pagi Darren,\n\nGreat news! We have ar...",1.0,request_onsite,Darren requests to book transport for two peop...,Confirm transportation booking for two people ...,send_onsite_request_to_staff,2024-04-23 02:50:42,2024-04-23 02:51:12,Message has been sent to staff. Message detail...,en
28,13465,HM4KT25MP5,Hosting,Selamat Pagi\n\nYes I would like to book trans...,"Selamat Pagi Darren,\n\nGreat news! We have ar...",1.0,request_onsite,Darren requests to book transport for two peop...,Confirm transportation booking for two people ...,send_onsite_request_to_staff,2024-04-23 02:50:42,2024-04-23 02:51:12,Message was not sent because it has been sent ...,en


In [26]:
# Truncate the strings in the specified column to the first 40 characters
column_name = 'information'
onsite_requests[column_name] = onsite_requests[column_name].astype(str).str[:50]

# Find the unique values in the truncated column
unique_values = onsite_requests[column_name].unique()

# Print or save the unique values
print(unique_values)

['Message has been sent to staff. Message details: 🇮'
 'Message was not sent because it has been sent to o'
 'Message has been sent to staff. Message details: P'
 'Message has been sent to staff. Message details: D'
 'Message has been sent to staff. Message details: 🧹'
 'Message has been sent to staff. Message details: M'
 'Message has been sent to staff. Message details: T'
 "{'error': True, 'data': {'booking_id': '53544167',"
 'Message has been sent to staff. Message details: L'
 "{'error': True, 'data': {'booking_id': 'ascascasca"
 'Message has been sent to staff. Message details: G'
 'Message has been sent to staff. Message details: K'
 'Message has been sent to staff. Message details: A'
 'Message has been sent to staff. Message details: R'
 'Message has been sent to staff. Message details: H'
 'Message has been sent to staff. Message details: S'
 "{'error': True, 'data': {'booking_id': 'HMYJPRTSBD"
 'Message has been sent to staff. Message details: B'
 'Message has been sent to s

C:\Users\ahini\AppData\Local\Temp\ipykernel_7152\3597891944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests[column_name] = onsite_requests[column_name].astype(str).str[:50]


In [44]:
# preprocess the 'last_message' column 
english_requests['last_message_clean'] = english_requests['last_message'].apply(preprocess_text)

C:\Users\ahini\AppData\Local\Temp\ipykernel_7152\680153648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_requests['last_message_clean'] = english_requests['last_message'].apply(preprocess_text)


In [45]:
english_requests.head()
columns = ['category_name', 'last_message_clean']
english_requests = english_requests[columns]

In [46]:
english_requests

,category_name,last_message_clean
3,request_onsite,"hi , ’ ant number 3 bathroom door ’ work got l..."
6,request_scooter_or_bike,"hi , would like know make scooter rental us ? ..."
7,request_scooter_or_bike,"hi , would like know make scooter rental us ? ..."
8,request_onsite,"hello , current someon staff properti . ceil leak"
9,request_onsite,"hello , could give key garag pleas"
...,...,...
3706,request_onsite,okey thank
3707,request_late_check_in,"great , thank much . lot harass “ taxi ” drive..."
3708,request_special_requirements,"hi , one cot pleas , sheet . ye , durat stay ...."
3709,request_special_requirements,"hi , one cot pleas , sheet . ye , durat stay ...."


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract features using TF-IDF
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(english_requests['last_message_clean'])

# find the top keywords for each category
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=english_requests['category_name'])

tfidf_sums = tfidf_df.groupby(tfidf_df.index).sum()

In [48]:
top_keywords = {}
for category in tfidf_sums.index:
    top_keywords[category] = tfidf_sums.loc[category].sort_values(ascending=False).head(10).index.tolist()
    
# Display top keywords for each category
for category, keywords in top_keywords.items():
    print(f"Category: {category}")
    print("Top keywords:", ", ".join(keywords))

Category: request_cancellation
Top keywords: refund, get, 12, reach, back, owner, talk, apart, surgeri, health
Category: request_early_check_in
Top keywords: thank, check, earli, possibl, around, much, arriv, hi, leav, earlier
Category: request_extension
Top keywords: extend, lack, shampoo, 19th, call, everyth, taxi, approv, help, room
Category: request_extra_service
Top keywords: go, us, lack, shampoo, hello, 19th, call, much, thank, everyth
Category: request_guest_size_alteration
Top keywords: car, villa, yo, adult, kawung, joglo, lunch, serv, deliv, might
Category: request_late_check_in
Top keywords: arriv, thank, check, hello, around, late, hi, flight, much, hour
Category: request_late_check_out
Top keywords: sit, pay, document, registr, pre, need, still, hope, wait, 30
Category: request_luggage_drop
Top keywords: leav, baggag, luggag, would, drop, in, thank, like, check, respons
Category: request_onsite
Top keywords: thank, hi, hello, much, arriv, water, pleas, room, check, come
C